# Trabajo 2 - IA

Resumen aqui


In [81]:
import kagglehub
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as stats
from sklearn.preprocessing import MinMaxScaler

In [82]:
path = kagglehub.dataset_download("neuralsorcerer/student-performance", path="test.csv")

df = pd.read_csv(path)
df = df.sample(n = 50_000, axis = 0, random_state=666)
df = df.reset_index(drop=True)

### Limpiando el Dataframe

In [83]:
df = df.drop(columns=['Extracurricular', 'PartTimeJob', 'ParentSupport', 'Romantic', 'Age'])
df.head()


,Grade,Gender,Race,SES_Quartile,ParentalEducation,SchoolType,Locale,TestScore_Math,TestScore_Reading,TestScore_Science,GPA,AttendanceRate,StudyHours,InternetAccess,FreeTime,GoOut
0,11,Male,White,1,SomeCollege,Public,City,99.098752,99.432820,89.003068,3.813771,0.980511,1.034361,0,4,3
1,9,Female,White,4,SomeCollege,Public,Rural,76.972814,73.094482,84.209950,3.423717,0.935273,0.885488,1,3,3
2,9,Female,Black,4,SomeCollege,Private,Suburban,72.286244,72.384595,76.809676,2.800215,0.912454,0.704442,1,1,2
3,12,Female,White,3,Bachelors+,Public,Rural,80.374680,72.682736,76.420886,2.707634,0.905701,1.122457,1,2,3
4,12,Female,Two-or-more,2,HS,Public,Suburban,76.024438,72.188542,79.180770,3.284770,0.906522,1.093387,0,4,4


In [84]:
# Inicializa las categorias para las variables categoricas que son numeros
categories = {1: "Very Low",2: "Low",3: "Moderate",4: "High", 5: "Very High"}

#Mapea los categorias
df["SES_Quartile"] = df["SES_Quartile"].map(categories).astype("category")
df["InternetAccess"] = df["InternetAccess"].map({0: "No",1: "Yes"}).astype("category")
df["FreeTime"] = df["FreeTime"].map(categories).astype("category")
df["GoOut"] = df["GoOut"].map(categories).astype("category")

#Convierte todo a categorias en vez de objetos
df[["Grade", "Gender", "Race", "ParentalEducation", "SchoolType", "Locale"]] = df[["Grade", "Gender", "Race", "ParentalEducation", "SchoolType", "Locale"]].astype("category")

#Se categoriza la variable objetivo en 4 clases
quartiles_GPA = df['GPA'].quantile([0.25, 0.5, 0.75])

bins = [0, quartiles_GPA[0.25], quartiles_GPA[0.5], quartiles_GPA[0.75], 5]
labels = ['Very Low', 'Low', 'Moderate', 'High']

df['GPA'] = pd.cut(df['GPA'], bins=bins, labels=labels, include_lowest=True)

#Insertar variable dependiente de primero por comodidad
gpa_serie = df['GPA']
df = df.drop(columns=['GPA'])
df.insert(0, 'GPA', gpa_serie)

In [ ]:
def create_df_variants(df: pd.DataFrame, outliers: bool = False, normalize: bool = False, balanced: bool = True) -> pd.DataFrame:
    def delete_outliers(df: pd.DataFrame):
        df_clean = df.copy()
        float_cols = df_clean.select_dtypes('float64').columns

        for col in float_cols:
            Q1 = df_clean[col].quantile(0.25)
            Q3 = df_clean[col].quantile(0.75)

            IQR = Q3 - Q1

            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]

        return df_clean

    def get_5outliers(df: pd.DataFrame):
        df_copy = df.copy()
        float_cols = df_copy.select_dtypes(include='float64').columns

        outlier_mask = pd.Series(False, index=df_copy.index)

        for col in float_cols:
            Q1 = df_copy[col].quantile(0.25)
            Q3 = df_copy[col].quantile(0.75)

            IQR = Q3 - Q1
            
            lower = Q1 - 1.5 * IQR
            upper = Q3 + 1.5 * IQR

            col_outliers = (df_copy[col] < lower) | (df_copy[col] > upper)
            outlier_mask |= col_outliers

        outliers = df_copy[outlier_mask]
        n_outliers = len(outliers)

        non_outliers = df_copy[~outlier_mask]
        sample_size = min(len(non_outliers), int(n_outliers * (1 - 0.05) / 0.05))
        sample = non_outliers.sample(n=sample_size, random_state=666)

        result_df = pd.concat([sample, outliers])

        return result_df

    def normalize_df(df: pd.DataFrame):
        df_norm = df.copy()
        float_cols = df_norm.select_dtypes(include='float64').columns

        scaler = MinMaxScaler()
        df_norm[float_cols] = scaler.fit_transform(df_norm[float_cols])

        return df_norm
    
    df_transformed = df.copy()

    # Con 5% outliers o sin ninguno
    df_transformed = get_5outliers(df_transformed) if outliers else delete_outliers(df_transformed)

    if normalize:
        df_transformed = normalize_df(df_transformed)

    return df_transformed





(49213, 16)
Outliers: 751 de 15019 → 5.00%
(15019, 16)
Outliers: 751 de 15019 → 5.00%
            GPA Grade  Gender      Race SES_Quartile ParentalEducation  \
22943  Very Low    11    Male  Hispanic         High        Bachelors+   
44370  Very Low    11  Female     White          Low       SomeCollege   
23906       Low     9    Male     White     Moderate       SomeCollege   
22308  Very Low     9    Male     Black         High                HS   
33181       Low    11    Male  Hispanic         High       SomeCollege   

      SchoolType    Locale  TestScore_Math  TestScore_Reading  \
22943     Public  Suburban       55.632817          49.154712   
44370     Public  Suburban       64.690828          72.281733   
23906     Public      City       78.462613          73.009873   
22308    Private  Suburban       65.748472          56.331438   
33181     Public      City       72.433603          87.211589   

       TestScore_Science  AttendanceRate  StudyHours InternetAccess  FreeTime 

In [86]:
df.head()

,GPA,Grade,Gender,Race,SES_Quartile,ParentalEducation,SchoolType,Locale,TestScore_Math,TestScore_Reading,TestScore_Science,AttendanceRate,StudyHours,InternetAccess,FreeTime,GoOut
0,High,11,Male,White,Very Low,SomeCollege,Public,City,99.098752,99.432820,89.003068,0.980511,1.034361,No,High,Moderate
1,High,9,Female,White,High,SomeCollege,Public,Rural,76.972814,73.094482,84.209950,0.935273,0.885488,Yes,Moderate,Moderate
2,Low,9,Female,Black,High,SomeCollege,Private,Suburban,72.286244,72.384595,76.809676,0.912454,0.704442,Yes,Very Low,Low
3,Low,12,Female,White,Moderate,Bachelors+,Public,Rural,80.374680,72.682736,76.420886,0.905701,1.122457,Yes,Low,Moderate
4,Moderate,12,Female,Two-or-more,Low,HS,Public,Suburban,76.024438,72.188542,79.180770,0.906522,1.093387,No,High,High
